In [1]:
# !wget https://huggingface.co/datasets/mesolitica/crawl-astroawani/resolve/main/berita-dunia-v2.json.nested
# !wget https://huggingface.co/datasets/mesolitica/crawl-astroawani/resolve/main/berita-hiburan-v2.json.nested
# !wget https://huggingface.co/datasets/mesolitica/crawl-astroawani/resolve/main/berita-malaysia-v2.json.nested
# !wget https://huggingface.co/datasets/mesolitica/crawl-astroawani/resolve/main/berita-politik-v2.json.nested
# !wget https://huggingface.co/datasets/mesolitica/crawl-astroawani/resolve/main/berita-sukan-v2.json.nested
# !wget https://huggingface.co/datasets/mesolitica/crawl-astroawani/resolve/main/berita-teknologi-v2.json.nested
# !wget https://huggingface.co/datasets/mesolitica/crawl-astroawani/resolve/main/gaya-hidup-v2.json.nested

In [2]:
import json
import malaya
from bs4 import BeautifulSoup
from glob import glob
from tqdm import tqdm
import random

/home/husein/dev/malaya/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3397
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/husein/dev/malaya/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3927
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))


In [3]:
files = sorted(glob('*.json.nested'))
files

['berita-dunia-v2.json.nested',
 'berita-hiburan-v2.json.nested',
 'berita-malaysia-v2.json.nested',
 'berita-politik-v2.json.nested',
 'berita-sukan-v2.json.nested',
 'berita-teknologi-v2.json.nested',
 'gaya-hidup-v2.json.nested']

In [4]:
from collections import defaultdict

In [5]:
texts = []
topics = defaultdict(set)

for f in files:
    print(f)
    with open(f) as fopen:
        data = json.load(fopen)
    for d in tqdm(data):
        if 'articleBody' not in d['r']['response']:
            continue
        try:
            soup = BeautifulSoup(d['r']['response']['articleBody'], "lxml")
            text = BeautifulSoup(soup.get_text(separator="\n").strip(), "lxml").get_text(separator="\n").strip()
            d['r']['response']['articleBody'] = text
            d['filename'] = f
            texts.append(d)
            tags = [t for t in d['original']['tags'] if len(t) > 5]
            topics[f].update(set(tags))
        except Exception as e:
            print(e)
            pass

berita-dunia-v2.json.nested


100%|███████████████████████████████████| 10000/10000 [00:03<00:00, 2707.17it/s]


berita-hiburan-v2.json.nested


100%|███████████████████████████████████| 10000/10000 [00:05<00:00, 1754.35it/s]


berita-malaysia-v2.json.nested


...

100%|███████████████████████████████████| 10000/10000 [00:04<00:00, 2232.22it/s]


berita-teknologi-v2.json.nested


100%|███████████████████████████████████████| 998/998 [00:00<00:00, 2106.05it/s]


gaya-hidup-v2.json.nested


100%|█████████████████████████████████████| 5366/5366 [00:03<00:00, 1701.39it/s]


In [6]:
topics_keys = set(topics.keys())

In [7]:
all_topics = list(topics.values())
len(all_topics)

7

In [8]:
flat_list = list(set([item for sublist in all_topics for item in sublist]))
len(flat_list)

77279

In [9]:
'a'.islower(), 'a'.count(' ')

(True, 0)

In [10]:
rejected = set([i for i in flat_list if i.islower() and i.count(' ') < 1])
len(rejected)

5094

In [11]:
for k in topics_keys:
    topics[k] = topics[k] - rejected

In [13]:
import random

X1, X2, Y = [], [], []

for d in tqdm(texts):
    if random.random() > 0.7:
        continue
        
    t = list(set(d['original']['tags']) - rejected)
    if not len(t):
        continue
    
    if random.random() > 0.5:
        X1.append(d['original']['title'])
        X2.append(f"topik {random.choice(t)}")
        Y.append(1)
    
    if random.random() > 0.5:
        X1.append(d['original']['description'])
        X2.append(f"topik {random.choice(t)}")
        Y.append(1)
    
    if random.random() > 0.5:
        X1.append(d['r']['response']['articleBody'])
        X2.append(f"topik {random.choice(t)}")
        Y.append(1)
        
print(len(Y))
for d in tqdm(texts):
    if random.random() > 0.8:
        continue
        
    t_n = set(random.choice(d['original']['tags']))
    t = list(topics[random.choice(list(topics_keys - {d['filename']}))] - t_n)
    if not len(t):
        continue
    
    if random.random() > 0.5:
        X1.append(d['original']['title'])
        X2.append(f"topik {random.choice(t)}")
        Y.append(0)
    
    if random.random() > 0.5:
        X1.append(d['original']['description'])
        X2.append(f"topik {random.choice(t)}")
        Y.append(0)
    
    if random.random() > 0.5:
        X1.append(d['r']['response']['articleBody'])
        X2.append(f"topik {random.choice(t)}")
        Y.append(0)
    
len(X1)

100%|█████████████████████████████████| 52548/52548 [00:00<00:00, 632912.51it/s]


54987


100%|███████████████████████████████████| 52548/52548 [00:16<00:00, 3143.90it/s]


118155

In [ ]:
with open('topics.json', 'w') as fopen:
    json.dump({
        'X1': X1,
        'X2': X2,
        'Y': Y,
    }, fopen)